# Optimal Execution of Trades in the FX Market

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm

sns.set(rc = {"figure.figsize":(16, 9)})

In [2]:
class OnlineData:

    def __init__(self, filename):
        self.filename = filename
        self.f = open(filename, 'r')
        self.TotalQ = 1
        self.Qnow = np.array([0])
        self.Pnow = np.array([0])
 
    def __next__(self):
        line = next(self.f)[:-1]
        if len(line) < 1:
            return None
        Q_, P_ = np.array([z[1:-1].split(",") for z in line.split("\t")
                          ]).astype(float).astype(int).T
        self.Qnow = Q_
        self.Pnow = P_
        CS = np.cumsum(P_ * np.ediff1d(Q_, to_begin=Q_[0]))

        self.TotalQ = sum(Q_)

        def F(Q):
            if Q <= Q_[0]:
                return P_[0] * Q
            if Q > Q_[-1]:
                return None
            i = np.searchsorted(Q_, Q)
            return CS[i - 1] + (Q - Q_[i - 1]) * P_[i]

        self.F = F
        return self.F

    def __iter__(self):
        return self

    def close(self):
        self.f.close()

In [3]:
def SrtategyMI(numbers: np.array, 
               times: np.array,
               orderbook_bid: OnlineData,
               dQfilename,
               marketIfilename):
        
        i = 0

        dQ = open(dQfilename, 'w+')
        marketI = open(marketIfilename, 'w+')

        for t in range(max(times) + 1):
            BuyPrice = next(orderbook_bid)

            if t in times:
                 deltaQ = numbers[i] / orderbook_bid.TotalQ * 10000
                 MarkI = ((BuyPrice(numbers[i] + 1) - BuyPrice(numbers[i])) / BuyPrice(1) - 1) * 10000
                 dQ.write(str(deltaQ) + " ")
                 marketI.write(str(MarkI) + " ")
                 print(f"time: {t} Ask: {BuyPrice(1)} BuyPrice: {BuyPrice(numbers[i]) / numbers[i]} MI: {MarkI}")
                 i += 1


        dQ.close()
        marketI.close()
                  

In [73]:
def OrderSizes(datafilename: str,
               dQfilename: str,
               len: int):
    
    it1 = OnlineData(datafilename)
    it2 = OnlineData(datafilename)

    next(it2)

    dQ = open(dQfilename, 'w+')

    for i in range(len):
        if(it1.Pnow[0] == it2.Pnow[0] and int(it1.Qnow[0]) == int(it2.Qnow[0])):
            next(it1)
            next(it2)
            continue
        else:
            if it1.TotalQ - it2.TotalQ > 5000000:
                dQ.write(str(i) + "\n")
            
            next(it1)
            next(it2)
        
    dQ.close()
            
        

In [3]:
def MIdQ(askfilename: str,
               dQfilename: str,
               MIfilename: str,
               depth: int):
    
    it1 = OnlineData(askfilename)
    it2 = OnlineData(askfilename)

    Price2 = next(it2)
    Price1 = next(it1)
    Price2 = next(it2)

    dQ = open(dQfilename, 'w+')
    MarkI = open(MIfilename, 'w+')
    Indexes = open("indexes.txt", 'w+')


    for i in range(depth):
        if(i > 100):
            volume = it1.TotalQ - it2.TotalQ
            deltaP = Price2(1) - Price1(1)
            if volume > 0 and not Price1(volume) == None and not deltaP == None and deltaP > 0:
                tildS = Price1(volume) / volume
                if(abs(tildS - Price1(1))/Price1(1) > 0):
                    MarkI.write(str(abs(tildS - Price1(1))/Price1(1) * 10000) + ", ")
                    dQ.write(str(volume / 142266100 * 10000) + ", ")
                    Indexes.write(str(i) + ", ")
                
        
        Price1 = next(it1)
        Price2 = next(it2)
        
    dQ.close()
            

In [33]:
dQs = np.genfromtxt("deltaQseries.txt", delimiter=", ", dtype = float)
print(len(dQs))

576


In [9]:
def MIdQ(askfilename: str,
               csvfilename: str,
               depth: int):
    
    it1 = OnlineData(askfilename)
    it2 = OnlineData(askfilename)

    Price2 = next(it2)
    Price1 = next(it1)
    Price2 = next(it2)

    cs = open(csvfilename, 'w+')
    cs.write("Time, Impact, DeltaQ\n")

    obnum = 1

    for i in range(depth):
        if(i > 1000):
            volume = it1.TotalQ - it2.TotalQ
            deltaP = Price2(1) - Price1(1)
            if volume > 0 and not Price1(volume) == None and not deltaP == None and deltaP > 0:
                tildS = Price1(volume) / volume
                if(abs(tildS - Price1(1))/Price1(1) > 0):
                    cs.write(str(float(i) / 50000) + ", " + str(abs(tildS - Price1(1))/Price1(1) * 10000) + ", " + str(volume / 142266100 * 10000) + ", \n")
                    obnum += 1
        
        Price1 = next(it1)
        Price2 = next(it2)


In [10]:
dQ = 'deltaQseries.txt'
dI = 'Iseries.txt'
dQs = np.array([0])
marketI = 'marketimpact.tsv'
ask = 'DataParsing/AllTheData/BidAsk/FullAsk.tsv'
bid = 'DataParsing/AllTheData/BidAsk/FullBid.tsv'

MIdQ(ask, "dQI.csv", 51000)
